In [17]:
# The code was removed by Watson Studio for sharing.

In [6]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


In [7]:
CLIENT_ID = '5A2XXCBQWE1YNEFR5B1FIR1LQ40EWRKA1YHTZO3GGBS3UFFI' # your Foursquare ID
CLIENT_SECRET = 'IHFJIG4OIPNF1PEL2NHQTT3DPY5RLX250V0LDQGB4MBO1F0K' # your Foursquare Secret
VERSION = '20200609'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5A2XXCBQWE1YNEFR5B1FIR1LQ40EWRKA1YHTZO3GGBS3UFFI
CLIENT_SECRET:IHFJIG4OIPNF1PEL2NHQTT3DPY5RLX250V0LDQGB4MBO1F0K


In [8]:
# Boston Coordinates (Right in front of City Hall)
latitude_boston = 42.360100
longitude_boston = -71.058900

# Seattle Coordinates (Next to Seattle Public Library)
latitude_seattle = 47.606200
longitude_seattle = -122.332100

radius = 5000
venue_limit = 500

company_category = 'Theater'

In [7]:
def next_venues_data(venue_id):
    
    url_next_venues = 'https://api.foursquare.com/v2/venues/{}/nextvenues?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result_next_venues = requests.get(url_next_venues).json()
    
    theater_prone = 0
    
    for next_venue in result_next_venues['response']['nextVenues']['items']:
        category = next_venue['categories'][0]['name']
        if category == company_category:
            theater_prone = 1
            break
    
    return theater_prone

In [24]:
venue_id = '4a4a438ff964a520a0ab1fe3'
url_next_venues = 'https://api.foursquare.com/v2/venues/{}/nextvenues?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result_next_venues = requests.get(url_next_venues).json()

result_next_venues['response']['nextVenues']['items']
print(next_venues_data(venue_id))

[0]


In [21]:
url_explore = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude_boston, longitude_boston, radius, venue_limit)
results = requests.get(url_explore).json()
results['response']['groups'][0]['items'][0]['venue']['id']

'4a4a438ff964a520a0ab1fe3'

# Check This for Venue Results

In [29]:
venue_id = '4a4a438ff964a520a0ab1fe3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
result

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5edfed1874cf2678a2122b59'},
 'response': {}}

In [25]:
def venues_data(latitude, longitude, radius, limit):
    
    url_explore = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, limit)
    results = requests.get(url_explore).json()
    items = results['response']['groups'][0]['items']
    
    venues_features = []
    
    for venue in items:
        venue_id = venue['venue']['id']
        venue_name = venue['venue']['name']
        venue_category = venue['venue']['categories'][0]['name']
        venue_distance = venue['venue']['location']['distance']

        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        result = requests.get(url).json()
        venue_info = result['response']['venue']
    
        venue_rating = venue_info['rating']
        venue_likes = venue_info['likes']['count']
        
        theater_prone = next_venues_data(venue_id)
        
        venues_features += [venue_id, venue_name, venue_category, venue_rating, venue_likes, venue_distance, theater_prone]
    
    return venues_features

In [27]:
print(venues_data(latitude_boston, longitude_boston, radius, venue_limit)[0:3])
url_explore = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, limit)
results = requests.get(url_explore).json()
items = results['response']['groups'][0]['items'][0:3]

KeyError: 'venue'

In [158]:
a = np.array([[1, 'a', 1], [2, 2, 2], [3, 3, 3]])
a.transpose()
a.dtype

dtype('<U21')

In [ ]:

venues_features_boston = venues_data(latitude_boston, longitude_boston, radius, venue_limit)
venues_data_boston = np.array(venues_features_boston).transpose

venues_features_seattle = venues_data(latitude_seattle, longitude_seattle, radius, venue_limit)
venues_data_seattle = np.array(venues_features_seattle).transpose

columns_labels = ['Id', 'Name', 'Category', 'Rating', 'Likes', 'Distance', 'Theater-Prone']

df_boston = pd.DataFrame(venues_data_boston, columns = columns_labels)
df_seattle = pd.DataFrame(venues_data_seattle, columns = columns_labels)


In [ ]:
from project_lib import Project
project = Project(project_id='09956828-79fb-4d57-b8be-8b60f3ecc367', project_access_token='p-6bd73bf226fe36ed663afb27af55ef287b59ea41')
project.save_data(file_name = "Boston_Venues_Data.csv", data = df_boston.to_csv(index=False))
project.save_data(file_name = "Seattle_Venues_Data.csv", data = df_seattle.to_csv(index=False))

In [ ]:
myfile = project.get_file("Boston_Venues_Data.csv")
venues_data_boston = pd.read_csv(myfile)
venues_data_boston.head()

In [3]:
a = np.array([[1,1], [2,2]])
b = np.array([[2], [1]])
c = np.append(a, b, axis = 1).transpose()
d = ['this', 'that']
dat = pd.DataFrame(c, columns = d)
dat.head()

,this,that
0,1,2
1,1,2
2,2,1


In [21]:
from project_lib import Project
project = Project(project_id='09956828-79fb-4d57-b8be-8b60f3ecc367', project_access_token='p-6bd73bf226fe36ed663afb27af55ef287b59ea41')
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df
project.save_data(file_name = "mypanda.csv",data = df.to_csv(index=False))

{'file_name': 'mypanda.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmdatasciencecourseracapstonepro-donotdelete-pr-glra0qtiffkarj',
 'asset_id': 'c7f404b7-89be-4a5f-9ade-5241c8e135c6'}

In [24]:
myfile = project.get_file("mypanda.csv")
test = pd.read_csv(myfile)
test

,A,B
0,1,2
1,3,4


In [2]:
# Creating DataFrame for the table found in the URL below
from lxml import html
import requests
from bs4 import BeautifulSoup



# Getting a request from the URL and making a BeautifulSoup object for the HTML source code
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(page.text, 'html.parser')

# Creating BeautifulSoup objects for the table for the DataFrame
soup_table = soup.find("table", class_="wikitable sortable").find_all('tr')


In [5]:
# Creating DataFrame for the table found in the URL below
from lxml import html
import requests
from bs4 import BeautifulSoup



# Getting a request from the URL and making a BeautifulSoup object for the HTML source code
page = requests.get('https://developer.foursquare.com/docs/build-with-foursquare/categories/')
soup = BeautifulSoup(page.text, 'html.parser')

# Creating BeautifulSoup objects for the table for the DataFrame
soup_list = soup.find("ul", class_="VenueCategories__Wrapper-sc-1ysxg0y-0 bYmzDC")


KeyboardInterrupt: 

In [ ]:
soup_header = soup_table[0].find_all('th')

# Creating a table for the DataFrame
table = np.empty((len(soup_table), len(soup_header))).astype('str')


for row_index, row in enumerate(soup_table):
    for column_index, column in enumerate(soup_header):
        table[row_index] = list(text for text in soup_table[row_index].stripped_strings)
        
table = table.transpose()
        
# Creating the DataFrame
df = pd.DataFrame({table[0][0]: table[0][1:],
                   table[1][0]: table[1][1:],
                   table[2][0]: table[2][1:]})
df.head()

In [40]:
url_categories = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)
result_categories = requests.get(url_categories).json()


In [35]:
result_categories['response']['categories'][0][]

{'id': '4d4b7104d754a06370d81259',
 'name': 'Arts & Entertainment',
 'pluralName': 'Arts & Entertainment',
 'shortName': 'Arts & Entertainment',
 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
  'suffix': '.png'},
 'categories': [{'id': '56aa371be4b08b9a8d5734db',
   'name': 'Amphitheater',
   'pluralName': 'Amphitheaters',
   'shortName': 'Amphitheater',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
    'suffix': '.png'},
   'categories': []},
  {'id': '4fceea171983d5d06c3e9823',
   'name': 'Aquarium',
   'pluralName': 'Aquariums',
   'shortName': 'Aquarium',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/aquarium_',
    'suffix': '.png'},
   'categories': []},
  {'id': '4bf58dd8d48988d1e1931735',
   'name': 'Arcade',
   'pluralName': 'Arcades',
   'shortName': 'Arcade',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/arcade_',
    

In [56]:
result_categories['response']['categories'][0]['categories']


[{'id': '56aa371be4b08b9a8d5734db',
  'name': 'Amphitheater',
  'pluralName': 'Amphitheaters',
  'shortName': 'Amphitheater',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
   'suffix': '.png'},
  'categories': []},
 {'id': '4fceea171983d5d06c3e9823',
  'name': 'Aquarium',
  'pluralName': 'Aquariums',
  'shortName': 'Aquarium',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/aquarium_',
   'suffix': '.png'},
  'categories': []},
 {'id': '4bf58dd8d48988d1e1931735',
  'name': 'Arcade',
  'pluralName': 'Arcades',
  'shortName': 'Arcade',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/arcade_',
   'suffix': '.png'},
  'categories': []},
 {'id': '4bf58dd8d48988d1e2931735',
  'name': 'Art Gallery',
  'pluralName': 'Art Galleries',
  'shortName': 'Art Gallery',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/artgallery_',
   'suffix': '.png'},
  'categor

In [91]:
def func(path, level, category, current_category):
    
    if level == 1:
        category = path[current_category]['name']
        
    if path[current_category]['categories'] == []:
        categories_dict[path[current_category]['name']] = category
        try:
            current_category += 1
            test_path = path[current_category]
            func(path, level, category, current_category)
        except:
            level -= 1
            func(path, level, category, current_category)
    else:
        level += 1
        path = path[current_category]['categories']
        func(path, level, category, 0)


In [92]:
categories_dict = {}

func(result_categories['response']['categories'], 0, [], 0)
    

In [93]:
categories_dict

{'Amphitheater': 'Amphitheater',
 'Aquarium': 'Aquarium',
 'Arcade': 'Arcade',
 'Art Gallery': 'Art Gallery',
 'Bowling Alley': 'Bowling Alley',
 'Casino': 'Casino',
 'Circus': 'Circus',
 'Comedy Club': 'Comedy Club',
 'Concert Hall': 'Concert Hall',
 'Country Dance Club': 'Country Dance Club',
 'Disc Golf': 'Disc Golf',
 'Exhibit': 'Exhibit',
 'General Entertainment': 'General Entertainment',
 'Go Kart Track': 'Go Kart Track',
 'Historic Site': 'Historic Site',
 'Karaoke Box': 'Karaoke Box',
 'Laser Tag': 'Laser Tag',
 'Memorial Site': 'Memorial Site',
 'Mini Golf': 'Mini Golf',
 'Drive-in Theater': 'Movie Theater',
 'Indie Movie Theater': 'Movie Theater',
 'Multiplex': 'Movie Theater'}

In [64]:
categories_dict['Movie Theater']

KeyError: 'Movie Theater'